# Лабораторная работа №5
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствитеёльности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [4]:
# Импортируем необходимые модули
from datetime import datetime as t
from collections import defaultdict

# Функция для алгоритма Кнута-Морриса-Пратта
def kmp_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text, pattern = text.lower(), pattern.lower()
    
    # Префикс-функция
    def compute_prefix_function(pattern):
        prefix = [0] * len(pattern)
        j = 0
        for i in range(1, len(pattern)):
            while j > 0 and pattern[i] != pattern[j]:
                j = prefix[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            prefix[i] = j
        return prefix

    prefix = compute_prefix_function(pattern)
    matches = []
    j = 0
    for i in range(len(text)):
        while j > 0 and text[i] != pattern[j]:
            j = prefix[j - 1]
        if text[i] == pattern[j]:
            j += 1
        if j == len(pattern):
            matches.append(i - j + 1)
            j = prefix[j - 1]
    return matches

# Функция для упрощенного алгоритма Бойера-Мура
def boyer_moore_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text, pattern = text.lower(), pattern.lower()
    
    def build_bad_character_table(pattern):
        table = defaultdict(lambda: -1)
        for i, char in enumerate(pattern):
            table[char] = i
        return table

    bad_char_table = build_bad_character_table(pattern)
    matches = []
    m, n = len(pattern), len(text)
    i = 0
    while i <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[i + j]:
            j -= 1
        if j < 0:
            matches.append(i)
            i += m - bad_char_table[text[i + m]] if i + m < n else 1
        else:
            i += max(1, j - bad_char_table[text[i + j]])
    return matches

# Функция для выделения подстроки в строке
def highlight_matches(text, pattern, matches):
    highlighted_text = ""
    last_index = 0
    for match in matches:
        highlighted_text += text[last_index:match] + "[" + text[match:match + len(pattern)] + "]"
        last_index = match + len(pattern)
    highlighted_text += text[last_index:]
    return highlighted_text

# Основная функция
def main():
    # Ввод данных
    text = input("Введите строку: ")
    pattern = input("Введите подстроку: ")
    case_sensitive = input("Чувствительность к регистру? (да/нет): ").strip().lower() == "да"

    # Замер времени и выполнение алгоритмов
    import time

    start = time.perf_counter()
    kmp_result = kmp_search(text, pattern, case_sensitive)
    kmp_time = time.perf_counter() - start

    start = time.perf_counter()
    bm_result = boyer_moore_search(text, pattern, case_sensitive)
    bm_time = time.perf_counter() - start

    start = time.perf_counter()
    python_result = [i for i in range(len(text)) if text[i:i+len(pattern)] == pattern] if case_sensitive else \
                    [i for i in range(len(text)) if text[i:i+len(pattern)].lower() == pattern.lower()]
    python_time = time.perf_counter() - start

    # Вывод результатов
    print("\nРезультаты поиска:")
    print(f"КМП: {kmp_result}, время: {kmp_time:.6f} сек")
    print(f"Бойер-Мур: {bm_result}, время: {bm_time:.6f} сек")
    print(f"Стандартный поиск: {python_result}, время: {python_time:.6f} сек")

    # Вывод строки с выделенной подстрокой
    if kmp_result:
        print("\nСтрока с выделенной подстрокой (КМП):")
        print(highlight_matches(text, pattern, kmp_result))
    if bm_result:
        print("\nСтрока с выделенной подстрокой (Бойер-Мур):")
        print(highlight_matches(text, pattern, bm_result))
    if python_result:
        print("\nСтрока с выделенной подстрокой (Стандартный поиск):")
        print(highlight_matches(text, pattern, python_result))

if __name__ == "__main__":
    main()


Результаты поиска:
КМП: [34], время: 0.000052 сек
Бойер-Мур: [34], время: 0.000043 сек
Стандартный поиск: [34], время: 0.000057 сек

Строка с выделенной подстрокой (КМП):
ывароллпаывполдлравывполдлравываро[аакк]рпвывпролопапрлдопавапроллопа

Строка с выделенной подстрокой (Бойер-Мур):
ывароллпаывполдлравывполдлравываро[аакк]рпвывпролопапрлдопавапроллопа

Строка с выделенной подстрокой (Стандартный поиск):
ывароллпаывполдлравывполдлравываро[аакк]рпвывпролопапрлдопавапроллопа


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [6]:
from collections import deque

# Проверка, является ли головоломка решаемой
def is_solvable(puzzle):
    inversion_count = 0
    puzzle = [num for num in puzzle if num != 0]  # Убираем пустую ячейку (0)
    for i in range(len(puzzle)):
        for j in range(i + 1, len(puzzle)):
            if puzzle[i] > puzzle[j]:
                inversion_count += 1
    return inversion_count % 2 == 0

# Генерация соседних состояний
def get_neighbors(state):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, 4)  # Размер поля 4x4

    # Возможные перемещения: вверх, вниз, влево, вправо
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for dr, dc in moves:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            new_zero_index = new_row * 4 + new_col
            new_state = state[:]
            # Меняем местами 0 и соседний элемент
            new_state[zero_index], new_state[new_zero_index] = new_state[new_zero_index], new_state[zero_index]
            neighbors.append((new_state, state[new_zero_index]))  # Сохраняем состояние и номер передвинутого элемента
    return neighbors

# Поиск решения с использованием BFS
def solve_puzzle(start):
    target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]  # Решенное состояние
    if start == target:
        return []  # Уже решено
    visited = set()
    queue = deque([(start, [])])  # Очередь: (состояние, путь)

    while queue:
        current_state, path = queue.popleft()
        visited.add(tuple(current_state))

        for neighbor, moved in get_neighbors(current_state):
            if tuple(neighbor) not in visited:
                if neighbor == target:
                    return path + [moved]  # Возвращаем путь к решению
                queue.append((neighbor, path + [moved]))
    return []  # Решения нет

# Основная функция
def main():
    # Ввод данных
    puzzle = list(map(int, input("Введите начальное состояние (16 чисел через пробел): ").split()))
    if len(puzzle) != 16 or set(puzzle) != set(range(16)):
        print("Некорректный ввод. Убедитесь, что ввели 16 чисел от 0 до 15.")
        return

    # Проверка решаемости
    if not is_solvable(puzzle):
        print("Данное расположение не является решаемым.")
        return

    # Поиск решения
    solution = solve_puzzle(puzzle)
    if solution:
        print("Решение найдено!")
        print("Последовательность движений:", solution)
    else:
        print("Решения нет.")

if __name__ == "__main__":
    main()

Данное расположение не является решаемым.


### Вывод